<img src="http://oci02.img.iteso.mx/identidad_de_instancia_2018/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg">

 ---

<center> <font color= #004A94> <font size = 4> Microestructura y Sistemas de Trading </font> <br> <br>

<center> <font color=#42c29b><font size=3> Notas: Herramientas Computacionales </font> <br> <br>

---

# Abstract

In [2]:
# -- Descargar precios OANDA
# -- Graficar OHLC con plotly
# -- Conteo de velas
# -- Informacion descriptiva general de EUR_USD

# Modulos

In [3]:
import funciones as fn                              # Para procesamiento de datos
import visualizaciones as vs                        # Para visualizacion de datos
import pandas as pd                                 # Procesamiento de datos
import numpy as np
import plotly.express as px
from datos import OA_Ak                             # Importar token para API de OANDA

# Desarrollo

Descarga Precios del servidor.

In [6]:
# -- --------------------------------------------------------- Descargar precios de OANDA -- #

# token de OANDA
OA_In = "EUR_USD"                  # Instrumento
OA_Gn = "D"                        # Granularidad de velas
fini = pd.to_datetime("2019-07-06 00:00:00").tz_localize('GMT')  # Fecha inicial
ffin = pd.to_datetime("2019-12-06 00:00:00").tz_localize('GMT')  # Fecha final

# Descargar precios masivos
df_pe = fn.f_precios_masivos(p0_fini=fini, p1_ffin=ffin, p2_gran=OA_Gn,
                             p3_inst=OA_In, p4_oatk=OA_Ak, p5_ginc=4900)

Grafica de Velas

In [7]:
# -- --------------------------------------------------------------- Graficar OHLC plotly -- #

vs_grafica1 = vs.g_velas(p0_de=df_pe.iloc[0:120, :])
vs_grafica1.show()


Conteo de Velas

In [8]:
# -- -------------------------------------------------------------------- Conteo de velas -- #

# multiplicador de precios
pip_mult = 10000

# -- 0A.1: Hora
df_pe['hora'] = [df_pe['TimeStamp'][i].hour for i in range(0, len(df_pe['TimeStamp']))]

# -- 0A.2: Dia de la semana.
df_pe['dia'] = [df_pe['TimeStamp'][i].weekday() for i in range(0, len(df_pe['TimeStamp']))]

# -- 0B: Boxplot de amplitud de velas (close - open).
df_pe['co'] = (df_pe['Close'] - df_pe['Open'])*pip_mult

In [9]:
# Para visualizar el estado actual del DataFrame con la información deseada. 
df_pe.head()

,TimeStamp,Open,High,Low,Close,hora,dia,co
0,2019-07-05 20:00:00+00:00,1.12264,1.12278,1.12247,1.12255,20,4,-0.9
1,2019-07-07 20:00:00+00:00,1.12248,1.12344,1.12071,1.12093,20,6,-15.5
2,2019-07-08 20:00:00+00:00,1.12095,1.12188,1.11935,1.12066,20,0,-2.9
3,2019-07-09 20:00:00+00:00,1.12070,1.12643,1.12020,1.12536,20,1,46.6
4,2019-07-10 20:00:00+00:00,1.12533,1.12858,1.12454,1.12570,20,2,3.7


Grafica de cajas y bigotes (Boxplot)

In [10]:
# -- ------------------------------------------------------------ Graficar Boxplot plotly -- #
vs_grafica2 = vs.g_boxplot_varios(p0_data=df_pe[['co']], p1_norm=False)
vs_grafica2.show()

Mes en el que ocurre la vela

In [11]:
# -- 01: Mes de la vela.
df_pe['mes'] = [df_pe['TimeStamp'][i].month for i in range(0,len(df_pe['TimeStamp']))]

Sesion en la que ocurre la vela (Europa, Asia, America, ...)

In [12]:
# -- 02: Sesion de la vela.
sesion = []
for i  in range(len(df_pe.mes)):
    if i in [22, 23, 0, 1, 2, 3, 4, 5, 6, 7]:
        s = 'asia'
    elif i in [8]:
        s = 'asia_europa'
    elif i in [9, 10, 11, 12]:
        s = 'europa'
    elif i in [13, 14, 15, 16]:
        s = 'europa_america'
    elif i in [17, 18, 19, 20, 21]:
        s = 'america'
    sesion.append(s)
df_pe['sesion'] = sesion

Amplitud entre precio de apertura y precio de cierre. 

In [13]:
# -- 03: Amplitud OC esperada de vela para cualquier dia de la semana (Dist de Freq).
df_pe['OC'] = (df_pe.Open - df_pe.Close)*pip_mult

Amplitud entre precio mínimo y máximo.

In [14]:
# -- 04: Amplitud HL esperada de vela para cualquier dia de la semana (Dist de Freq).
df_pe['hl'] = (df_pe.High - df_pe.Low)*pip_mult

Evolución de las velas. Consiste en dos partes, la primera calcular si es alcista o bajista,
la segunda en contar la cantidad de velas seguidas. 

In [15]:
# -- 05: Evolucion de velas consecutivas (1: Alcistas, 0: Bajistas).
df_pe['sentido'] = ['alcista'if i>=0 else 'bajista' for i in df_pe.co]
df_pe['sentido_c'] = fn.secuencial(df_pe.sentido)

Máxima diferencia entre precio mínimo y máximo de los últimos 5, 25 y 50 días pasados

In [16]:
# -- 06: Maxima evolucion esperada de velas consecutivas (Dist Acum de Freq).
periodos = [5, 25, 50]
tmp = [[df_pe['hl'][i:i+5].max() for i in range(len(df_pe.hl)-j)] for j in periodos]
volatilidad_5 = tmp[0]
volatilidad_25 = tmp[1]
volatilidad_50 = tmp[2]

Graficas: 
- De diferencias entre apertura y cierre de precios, expresada en pips.
- De volatilidad en los últimos 25 días. 

In [17]:
# -- 07: Calculo + Grafica autopropuesta.
px.line(df_pe, x = "TimeStamp", y = "OC", title = 'Open-Close pips').show()

In [21]:
fig = px.line(x = np.arange(len(volatilidad_25)), y = volatilidad_25)
fig.update_layout(
    title="Volatilidad 25",
    xaxis_title="tiempo",
    yaxis_title="max_diff_OC")
fig.show()